In [1]:
!pip install pandas_ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=4ae12a739747b6b7f28673ecca969a042e5a1762b1b76522b87187af97440233
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
pip install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git


  Cloning https://github.com/rongardF/tvdatafeed.git to /tmp/pip-req-build-73ux66a0
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed.git /tmp/pip-req-build-73ux66a0
  Resolved https://github.com/rongardF/tvdatafeed.git to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=343eb63bd206604dc5176a4c43549ef347ef3b3d320f969a6caa98f9c8c2f30e
  Stored in directory: /tmp/pip-ephem-wheel-cache-75yxctuu/wheels/4b/f9/e9/82474a5d9a64702ba7dd5481dee05d267c69bcfa480d382b7c
Successfully built tvdatafeed


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tvDatafeed import TvDatafeed, Interval

#import yfinance as yf
import pandas_ta as ta
tv = TvDatafeed()


In [4]:
# Adding indicators

hal_df=tv.get_hist(symbol='GESHIP',exchange='NSE',interval=Interval.in_15_minute,n_bars=5000)
data=pd.DataFrame(hal_df)

data['RSI']=ta.rsi(data.close, length=3)
data['EMAF']=ta.ema(data.close, length=8)
data['EMAm']=ta.ema(data.close, length=13)

data['EMAs']=ta.ema(data.close, length=21)
data['EMASs']=ta.ema(data.close, length=44)

data['Adj Close'] = data['close'].shift(+1)
data['Target'] = data['Adj Close']-data.open
data['Target'] = data['Target'].shift(-1)

data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]

data['TargetNextClose'] = data['close'].shift(-1)

data.dropna(inplace=True)
data.reset_index(inplace = True)
data.drop(['volume', 'datetime','symbol','high','low'], axis=1, inplace=True)
data.head(5)
len(data)


4956

In [5]:
data_set = data.iloc[:, 0:11]#.values
pd.set_option('display.max_columns', None)


In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)


In [7]:
X = []
backcandles = 30
print(data_set_scaled.shape[0])
for j in range(8):#data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]):#backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

X=np.moveaxis(X, [0], [2])
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))

print(X.shape)
print(y.shape)
len(X)

4956
(4926, 30, 8)
(4926, 1)


4926

In [8]:
# split data into train test sets
splitlimit = 4920
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:splitlimit+1]
y_train, y_test = y[:splitlimit], y[splitlimit:splitlimit+1]
#y_predict,xpredict=y[splitlimit+1:splitlimit+2],X[splitlimit+1:splitlimit+2]
buy_price=X_test[0,9,0]
print(buy_price)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
#print(X_test)
print(y_test)
#y_predict

4920
0.9027898866608542
(4920, 30, 8)
(1, 30, 8)
(4920, 1)
(1, 1)
[[0.83727312]]


In [9]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)

lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
inputs = LSTM(150, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train, y=y_train, batch_size=15, epochs=5, shuffle=True, validation_split = 0.1)

Epoch 1/5
296/296 [==============================] - 26s 63ms/step - loss: 0.0013 - val_loss: 6.5347e-04
Epoch 2/5
296/296 [==============================] - 11s 39ms/step - loss: 1.4210e-04 - val_loss: 5.9757e-04
Epoch 3/5
296/296 [==============================] - 10s 35ms/step - loss: 1.3330e-04 - val_loss: 7.3502e-04
Epoch 4/5
296/296 [==============================] - 10s 34ms/step - loss: 1.4154e-04 - val_loss: 6.7233e-04
Epoch 5/5
296/296 [==============================] - 12s 40ms/step - loss: 1.3690e-04 - val_loss: 6.0248e-04


In [10]:
y_pred = model.predict(X_test)
#y_pred=np.where(y_pred > 0.43, 1,0)
for i in range(1):
    print('y_test[i]',y_test[0,0])
    print('y_pred[i]',y_pred[0])

'''y_pred[0].reshape(1,-1)
y_test[0].reshape(1,-1)'''
reshaped_array = np.zeros(( 1,11))
reshaped_array1 = np.zeros(( 1,11))
reshaped_array2 = np.zeros(( 1,11))


print(reshaped_array)
reshaped_array[0,0]=y_pred[0]
print(reshaped_array)
#
reshaped_array1[0,0]=y_test[0,0]
reshaped_array2[0,0]=buy_price

y_pred = sc.inverse_transform(reshaped_array)
print('y_pred:',y_pred)
y_test=sc.inverse_transform(reshaped_array1)
print('y_test:',y_test)
buy=sc.inverse_transform(reshaped_array2)
print('y_test:',y_test[0,0])
print('y_pred:',y_pred[0,0])
print('buy_price:',buy[0,0])
if y_pred[0,0]>buy[0,0]:

   print('profit/loss:',y_test[0,0]-buy[0,0])
   print('profit/loss %:',((y_test[0,0]-y_pred[0,0])/buy[0,0]))
else:
   print('profit/loss:',(buy[0,0]-y_test[0,0]))
   print('profit/loss %:',((buy[0,0]-y_pred[0,0])/buy[0,0]))




1/1 [==============================] - 0s 462ms/step
y_test[i] 0.8372731241813758
y_pred[i] [0.83231586]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.83231586 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]]
y_pred: [[ 1.39726641e+03  7.29000000e+02  7.98875755e-02  7.30204297e+02
   7.31945124e+02  7.35280877e+02  7.43942140e+02  7.29000000e+02
  -6.00000000e+01  0.00000000e+00  7.29000000e+02]]
y_test: [[ 1.40124659e+03  7.29000000e+02  7.98875755e-02  7.30204297e+02
   7.31945124e+02  7.35280877e+02  7.43942140e+02  7.29000000e+02
  -6.00000000e+01  0.00000000e+00  7.29000000e+02]]
y_test: 1401.2465914052268
y_pred: 1397.2664057433606
buy_price: 1453.85
profit/loss: 52.6034085947731
profit/loss %: 0.03891982959496464


<ipython-input-10-34aba8988573>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  reshaped_array[0,0]=y_pred[0]


In [11]:
if y_pred[0,0]>buy[0,0]:

   print('profit/loss %:',((y_test[0,0]-y_pred[0,0])/buy[0,0])*100)
else:
   print('profit/loss %:',((buy[0,0]-y_pred[0,0])/buy[0,0])*100)


profit/loss %: 3.891982959496464
